In [ ]:
def bootstrap_analysis_ridge_lambas(nlambdas = 10):
    np.random.seed(10)
    lambdas = np.logspace(-4,4, nlambdas)
    mse_aggregate = np.zeros([3, nlambdas])
    best_complexities = np.zeros([3,nlambdas])
    for data_point_index, data_points in enumerate([20, 50, 100]):
        print(f"Number of datapoints {data_points}")
        global_best_mse = float('inf')
        global_best_complexity = 0
        best_num_bstraps = 0
        best_train_split = 0
        best_lamb = 0
        for lamb_ind, lamb in enumerate(lambdas):
            best_mse, best_complexity = bias_variance_analysis_bootstrap(Ridge, data_points, max_degree=10, num_bootstraps = data_points, lamb = lamb, plot_log = False, plot = False)
            #print(f"Lambda: {lamb}. Datapoints: {data_points}. MSE: {best_mse}")
            if best_mse < global_best_mse:
                global_best_mse = best_mse
                global_best_complexity = best_complexity
                best_num_bstraps = data_points
                best_lamb = lamb
            
            mse_aggregate[data_point_index, lamb_ind] = best_mse
            best_complexities[data_point_index, lamb_ind] = best_complexity
                
        print(f"Best lambda: {best_lamb} Best global: {global_best_mse} at complexity: {global_best_complexity} for number of datapoints: {data_points}, num_bootstraps = {data_points}")
        print(bias_variance_analysis_bootstrap(Ridge, data_points, max_degree=10, num_bootstraps = best_num_bstraps, plot=True, plot_log=False,  lamb = best_lamb))
        
    return mse_aggregate, lambdas, best_complexities


def plot_lambda_dependency():
    mse_aggregate, lambdas, complexity = bootstrap_analysis_ridge_lambas(100)
    for i, data_points in enumerate(mse_aggregate):
        fig, ax1 = plt.subplots()
        color = 'tab:red'
        ax1.set_xlabel('log10(lambda)')
        ax1.set_ylabel('mse')
        ax1.plot(np.log10(lambdas), data_points, label = 'MSE per lambda', color = color)
        ax1.tick_params(axis='y', labelcolor=color)
        
        ax2 = ax1.twinx()
        color = 'tab:blue'
        ax2.set_ylabel('Complexity', color=color)
        ax2.plot(np.log10(lambdas), complexity[i], color=color, label="Best Complexity")
        ax2.tick_params(axis='y', labelcolor=color)

        fig.tight_layout()  # otherwise the right y-label is slightly clipped    
        plt.title(f"MSE per lambda for {len(data_points)} datapoints")
        ax1.legend()
        ax2.legend()
        plt.show()

plot_lambda_dependency()

In [ ]:
def cv_analysis_ridge_lambas(num_points, min_fold, max_fold, max_complecity, nlambdas = 10, plot = False, show_plot = False):
    np.random.seed(88)
    lambdas = np.logspace(-4,4, nlambdas)
    for lamb in lambdas:
        plt.figure(figsize=(20,5))
        for fold in range(min_fold, max_fold+1):
            cross_validation(Ridge, num_points, fold, max_complecity, lamb=lamb, plot=plot)

        if show_plot:
            plt.title(f"Lambda: {lamb}")
            plt.show()
cv_analysis_ridge_lambas(40, 5, 10, 10,  nlambdas = 100, plot= True, show_plot = True)

#### NOTE: The complexity of the different lambdas may vary. This is just a plot of the absolute best complexity of each of the lambdas.

In [ ]:
def simple_ridge_run(
    x = (np.random.uniform(0, 1, 1000)), 
    y =  (np.random.uniform(0, 1, 1000)), 
    lamb = 0,
    num_points = 1000, 
    complexity = 5, 
    noise = 0, 
    scale = True, 
    plot_mse = False, 
    plot_r2 = False):
    """
    Computes the simples ordinary least square based on the Franke Function
    
    Args:
        stuff
        
    Returns:
        ols_beta: The OLS
    """
    
    if num_points != len(x):
        x = (np.random.uniform(0, 1, num_points))
        y =  (np.random.uniform(0, 1, num_points))
        
        
    MSE_train = []
    MSE_pred = []
    r2_train = []
    r2_pred = []
    
    all_ols_betas = []
    all_xtx_inv = []

    for complexity in range(1,complexity+1):

        #Trying not to sort the x and y's
        z = FrankeFunction(x, y, noise) # Target
        X = create_X(x, y, n=complexity)  # Data

        # True to z instead of y, and same with predictions: z_pred instead of y_pred
        X_train, X_test, z_train, z_test = train_test_split(X, z, test_size=0.2)
        #scaler = MinMaxScaler(feature_range= [-1,1])
        scaler_in = StandardScaler(with_std=False)
        scaler_in.fit(X_train)
        scale_z = StandardScaler(with_std=False)
        scale_z.fit(z_train)
        
# Ridge: fit_intesect = False, da bryr vi oss ikke om intersect

        if scale:
            X_train = scaler_in.transform(X_train)
            X_test = scaler_in.transform(X_test)
            #X_train -= np.mean(X_train)
            #X_test -= np.mean(X_test)
            z_train = scale_z.transform(z_train)
            z_test = scale_z.transform(z_test)


        ols_beta = Ridge(X_train, z_train, lamb)
        all_ols_betas.append(ols_beta)
        
        xtx = np.linalg.pinv(X_train.transpose().dot(X_train))
        all_xtx_inv.append(xtx)

        z_tilde = X_train.dot(ols_beta)
        z_pred = X_test.dot(ols_beta)


        mse_train = mean_squared_error(z_tilde, z_train)
        MSE_train.append(mse_train)
        mse_test = mean_squared_error(z_pred, z_test)
        MSE_pred.append(mse_test)

        r2_train.append(r2_score(z_tilde, z_train))
        r2_pred.append(r2_score(z_pred, z_test))
    
    if plot_mse:
        plot_errors(
            x_range_train = np.arange(1, complexity+1), 
            x_range_test = np.arange(1, complexity+1), 
            y_values_train = MSE_train, 
            y_values_test = MSE_pred,
            title = 'MSE by complexity. Lambda = '+str(lamb), 
            xlabel_axis = 'Complexity',
            ylabel_axis = 'MSE',
            graph_label_train = 'mse_train',
            graph_label_test = 'mse_test'
        )

    if plot_r2:
        plot_errors(
            x_range_train = np.arange(1, complexity+1), 
            x_range_test = np.arange(1, complexity+1), 
            y_values_train = r2_train, 
            y_values_test = r2_pred,
            title = 'R2 by complexity. Lambda = '+str(lamb), 
            xlabel_axis = 'Complexity',
            ylabel_axis = 'R2 score',
            graph_label_train = 'r2_train',
            graph_label_test = 'r2_test'
        )
    
    
    return all_ols_betas, all_xtx_inv


In [ ]:
def simple_mse_and_r2_by_complexity(
    reg_func = least_square,
    x = (np.random.uniform(0, 1, 1000)), 
    y =  (np.random.uniform(0, 1, 1000)), 
    num_points = 1000, 
    complexity = 5, 
    noise = 0, 
    scale = True, 
    plot_mse = False, 
    plot_r2 = False,
    y_scale = 'linear',
    lamb = 0):
    """
    Computes the simples ordinary least square based on the Franke Function
    
    Args:
        stuff
        
    Returns:
        ols_beta: The OLS
    """
    
    if num_points != len(x):
        x = (np.random.uniform(0, 1, num_points))
        y =  (np.random.uniform(0, 1, num_points))
        
        
    MSE_train = []
    MSE_pred = []
    r2_train = []
    r2_pred = []
    
    all_ols_betas = []
    all_xtx_inv = []
    
    lass_mse_train = []
    lass_mse_test = []
    lass_r2_train = []
    lass_r2_test = []

    for complexity in range(1,complexity+1):

        #Trying not to sort the x and y's
        z = FrankeFunction(x, y, noise) # Target
        X = create_X(x, y, n=complexity)  # Data

        # True to z instead of y, and same with predictions: z_pred instead of y_pred
        X_train, X_test, z_train, z_test = train_test_split(X, z, test_size=0.2)
        #scaler = MinMaxScaler(feature_range= [-1,1])
        scaler_in = StandardScaler(with_std=False)
        scaler_in.fit(X_train)
        scale_z = StandardScaler(with_std=False)
        scale_z.fit(z_train)
        
        if scale:
            X_train = scaler_in.transform(X_train)
            X_test = scaler_in.transform(X_test)
            #X_train -= np.mean(X_train)
            #X_test -= np.mean(X_test)
            z_train = scale_z.transform(z_train)
            z_test = scale_z.transform(z_test)


        beta_opt = reg_func(X_train, z_train, lamb)
        all_ols_betas.append(beta_opt)
        
        xtx = np.linalg.pinv(X_train.transpose().dot(X_train))
        all_xtx_inv.append(xtx)

        z_tilde = X_train.dot(beta_opt)
        z_pred = X_test.dot(beta_opt)


        mse_train = mean_squared_error(z_tilde, z_train)
        MSE_train.append(mse_train)
        mse_test = mean_squared_error(z_pred, z_test)
        MSE_pred.append(mse_test)

        r2_train.append(r2_score(z_tilde, z_train))
        r2_pred.append(r2_score(z_pred, z_test))
        
        lasso = linear_model.Lasso(fit_intercept = False, alpha = lamb, max_iter = 10000, tol = 0.005)
        lasso.fit(X_train, z_train)
        z_tilde = lasso.predict(X_train)
        z_pred = lasso.predict(X_test)
        mse_train = mean_squared_error(z_tilde, z_train)
        lass_mse_train.append(mse_train)
        mse_test = mean_squared_error(z_pred, z_test)
        lass_mse_test.append(mse_test)

        lass_r2_train.append(r2_score(z_tilde, z_train))
        lass_r2_test.append(r2_score(z_pred, z_test))
        
    
    if plot_mse:
        plt.figure(figsize = (20,5))
        plot_errors(
            x_range_train = np.arange(1, complexity+1), 
            x_range_test = np.arange(1, complexity+1), 
            y_values_train = MSE_train, 
            y_values_test = MSE_pred,
            title = 'MSE by complexity', 
            xlabel_axis = 'Complexity',
            ylabel_axis = 'MSE',
            graph_label_train = 'mse_train',
            graph_label_test = 'mse_test',
            y_scale = y_scale
        )
        
        
        plot_errors(
            x_range_train = np.arange(1, complexity+1), 
            x_range_test = np.arange(1, complexity+1), 
            y_values_train = lass_mse_train, 
            y_values_test = lass_mse_test,
            title = 'MSE by complexity', 
            xlabel_axis = 'Complexity',
            ylabel_axis = 'MSE',
            graph_label_train = 'mse_train_lasso',
            graph_label_test = 'mse_test_lasso',
            y_scale = y_scale
        )
        plt.show()

    if plot_r2:
        plt.figure(figsize = (20,5))
        plot_errors(
            x_range_train = np.arange(1, complexity+1), 
            x_range_test = np.arange(1, complexity+1), 
            y_values_train = r2_train, 
            y_values_test = r2_pred,
            title = 'R2 by complexity', 
            xlabel_axis = 'Complexity',
            ylabel_axis = 'R2 score',
            graph_label_train = 'r2_train',
            graph_label_test = 'r2_test'
        )
        
        plot_errors(
            x_range_train = np.arange(1, complexity+1), 
            x_range_test = np.arange(1, complexity+1), 
            y_values_train = lass_r2_train, 
            y_values_test = lass_r2_test,
            title = 'R2 by complexity', 
            xlabel_axis = 'Complexity',
            ylabel_axis = 'R2 score',
            graph_label_train = 'r2_train_lasso',
            graph_label_test = 'r2_test_lasso'
        )
        plt.show()
    
    
    return all_ols_betas, all_xtx_inv

betas, xtx = simple_mse_and_r2_by_complexity(num_points = 1000, complexity = 20, noise = 0.1, plot_mse = True, plot_r2 = True)


### Bias-variance trade-off without bootstrap with constant data points

We see that we get no variance, and that the bias = error.

In [3]:
import os
import os.path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.utils import resample
from scipy.stats import norm
from sklearn.exceptions import ConvergenceWarning
from sklearn.utils.testing import ignore_warnings

from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

from imageio import imread
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm

np.random.seed(np.random.randint(1,1000))

C:\Users\willi\anaconda3\Lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.utils.testing module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.utils. Anything that cannot be imported from sklearn.utils is now part of the private API.
  warnings.warn(message, FutureWarning)


In [16]:
def load_data(file_name: str = "SRTM_data_Norway_1.tif", re_size = 10, display: bool = False):
    """Load terrain dataset from image and create x and y coordinates.
    Returns
    -------
    x, y, z
    """
    data = imread("./data/SRTM_data_Norway_1.tif")

    if display:
        plt.imshow(data, plt.cm.gray)
        plt.axis("off")
        plt.show()
        
    np_terrain1 = np.array(data)
    x = np.linspace(0, 1, np_terrain1.shape[0])
    y = np.linspace(0, 1, np_terrain1.shape[1])
    x, y = np.meshgrid(x, y)
    
    x = x[0::2*re_size, 0::re_size].reshape(-1,1)
    y = y[0::2*re_size, 0::re_size].reshape(-1,1)
    data = data.T[0::2*re_size, 0::re_size].reshape(-1,1) #Transpose, because because 

    return x, y, data
x,y,data = load_data()
x.shape, y.shape, data.shape
create_X(x,y, 1).shape #(65341, 3)

(32851, 3)

In [12]:
def FrankeFunction(x, y, sigma = 0):
    term1 = 0.75*np.exp(-(0.25*(9*x-2)**2) - 0.25*((9*y-2)**2))
    term2 = 0.75*np.exp(-((9*x+1)**2)/49.0 - 0.1*(9*y+1))
    term3 = 0.5*np.exp(-(9*x-7)**2/4.0 - 0.25*((9*y-3)**2))
    term4 = -0.2*np.exp(-(9*x-4)**2 - (9*y-7)**2)

    noise = np.random.normal(0, 1, x.shape[0])
    noise = noise.reshape(x.shape[0],1)

    return (term1 + term2 + term3 + term4).reshape(-1,1)  + sigma*noise

def create_X(x, y, n ):
    if len(x.shape) > 1:
        x = np.ravel(x)
        y = np.ravel(y)

    N = len(x)
    l = int((n+1)*(n+2)/2)   # Number of elements in beta
    X = np.ones((N,l))

    for i in range(1,n+1):
        q = int((i)*(i+1)/2)
        for k in range(i+1):
            X[:,q+k] = (x**(i-k))*(y**k)

    return X

def least_square(x_value,y_value, *args, **kwargs):
    # Using pinv
    return np.linalg.pinv(x_value.transpose().dot(x_value)).dot(x_value.transpose().dot(y_value))

def plot_errors(x_range_train, x_range_test, y_values_train, y_values_test, title, xlabel_axis, ylabel_axis, graph_label_train, graph_label_test, y_scale = 'linear'):
    fig = plt.figure(figsize = (20,5))
    plt.xlabel(xlabel_axis)
    plt.ylabel(ylabel_axis)
    plt.title(title)
    plt.yscale(y_scale)
    plt.plot(x_range_train, y_values_train, label=graph_label_train)
    plt.plot(x_range_test, y_values_test, label=graph_label_test)
    plt.legend()
    plt.show()


In [ ]:
def bias_variance_analysis(reg_func, num_points, max_degree, lamb = 0):
    #np.random.seed(88)
    # Make data
    x = (np.random.uniform(0, 1, num_points))
    y =  (np.random.uniform(0, 1, num_points))
    z = FrankeFunction(x, y, 0.4) # Target
    def load_data(file_name: str = "SRTM_data_Norway_1.tif", re_size = 0, display: bool = False) -> tuple[np.ndarray, ...]:
    """Load terrain dataset from image and create x and y coordinates.
    Returns
    -------
    x, y, z
    """
    image_path = os.path.join(
        os.path.dirname(__file__),
        file_name,
    )
    data = imageio.imread(image_path)

    if display:
        plt.imshow(data, plt.cm.gray)
        plt.axis("off")
        plt.show()
        
    np_terrain1 = np.array(terrain1)
    x = np.linspace(0, 1, np_terrain1.shape[0])
    y = np.linspace(0, 1, np_terrain1.shape[1])
    x, y = np.meshgrid(x, y)
    
    x = x[0::2*10, 0::10].reshape(-1,1)
    y = y[0::2*10, 0::10].reshape(-1,1)
    data = data.T[0::2*10, 0::10].reshape(-1,1) #Transpose, because because 

    return x, y, data

    # Aggregrate results
    error = np.zeros(max_degree)
    bias = np.zeros(max_degree)
    variance = np.zeros(max_degree)
    
    for complexity in range(max_degree):
        
        # Make design matrix
        X = create_X(x, y, n=complexity)
        
        # Split training data
        X_train, X_test, z_train, z_test = train_test_split(X, z, test_size=0.2)
        # Scaling the data
        """
        scaler_in = StandardScaler(with_std=False)
        scaler_in.fit(X_train)
        scale_z = StandardScaler(with_std=False)
        scale_z.fit(z_train)
        
        X_train = scaler_in.transform(X_train)
        X_test = scaler_in.transform(X_test)
        z_train = scale_z.transform(z_train)
        z_test = scale_z.transform(z_test)
        """
        # Find optimal beta
        beta_opt = reg_func(X_train, z_train, lamb)
        
        # Predict
        z_pred = X_test.dot(beta_opt)
        
        # Loss:
        mse_test = mean_squared_error(z_pred, z_test)
        
        # Aggregrate stats:
        error[complexity] = mse_test
        bias[complexity] = np.mean((z_test - np.mean(z_pred, axis=1, keepdims=True))**2)
        variance[complexity] = np.mean((z_pred - np.mean(z_pred, axis = 1, keepdims = True))**2)
        
    polydegree =  np.arange(max_degree)
    plt.plot(polydegree, error, label='Error')
    plt.plot(polydegree, bias, label='bias')
    plt.plot(polydegree, variance, label='Variance')
    plt.xlabel("Complexity")
    plt.ylabel("Error")
    
    plt.legend()
    plt.show()
    
    """
    stuff = np.add(bias,variance)
    print(variance)
    print(error - stuff)
    #print(variance)
    #print(np.mean(z_pred, axis = 1, keepdims = True))
    #print(z_pred.shape)
    #print(np.abs(z_pred-np.mean(z_pred, axis = 0, keepdims = True)))
    """
    
bias_variance_analysis(least_square, 100, 10)